In [12]:
import torch
import d2l as d2l


batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

每个样本都会用固定长度的向量表示，原始数据集的每个样本都是28×28的图像，展开后可以看作长度为784的向量。

softmax回归中，输出和类别一样多，因此每个样本有10个输出，所以权重会构成一个784×10的矩阵，偏置是一个1×10的向量。

In [3]:
num_input = 784
num_output = 10

w = torch.normal(0, 0.01, size=(num_input, num_output), requires_grad=True)
b = torch.zeros(num_output,requires_grad=True)

定义softmax操作，softmax有3个步骤组成：
* 对每个项求幂（exp）
* 对每一行求和，得到每个样本的归一化常数
* 将每一行除以其归一化常数，确保结果和为1

softmax函数的意义是让是个输出均为正数并且和为1，这样复合统计学中的概率分布。

In [4]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition

In [5]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0606, 0.0198, 0.6801, 0.0272, 0.2123],
         [0.1598, 0.0942, 0.0715, 0.3823, 0.2922]]),
 tensor([1., 1.]))

定义softmax模型

In [6]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, w.shape[0])), w) + b)

定义损失函数

选用交叉熵函数作为损失函数，因此只需要选择样本真实的标签来求损失,使用的是真是标签的负对数似然。

In [7]:
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

分类准确率，即正确预测数量与总预测数量之比。

In [8]:
def accuracy(y_hat, y):
    """ 计算预测正确的数量 """
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)    # 类似于sum中的参数keepdim
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

对于任意数据迭代器data_iter可访问的数据集，可以评估在任意模型net的准确率。

In [9]:
class Accumulator:  #@save
    """ 在n个变量上累加 """
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]


def evaluate_accuracy(net, data_iter):  #@save
    """ 计算在指定数据集上模型的精度 """
    if isinstance(net, torch.nn.Module):
        net.eval()  # 将模型设为评估模式
    metric = Accumulator(2)  # 创建两个变量，一个存储正确预测数量，一个存储总数量
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


evaluate_accuracy(net, test_iter)

0.1194

训练

In [11]:
def train_epoch_ch3(net, train_iter, loss, updater):  #@save
    """ 训练一个模型的迭代周期 """
    # 设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用Pytorch内置的优化器和损失函数
            updater.zero_grad()
            l.backwarrd()
            updater.step()
            metric.add(float(l) * len(y), accuracy(y_hat, y), y.size().numel())
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    # 返回训练损失和训练准确率
    return metric[0] / metric[2], metric[1] / metric[2]


def train_ch3(net, train_iter, test_iter, loss, num_epoch3, updater):
    """ 训练模型 """
    for epoch in range(num_epoch3):
        train_metrics = train_epoch_ch3(net, train_iter, loss, updater)
        test_acc = evaluate_accuracy(net, test_iter)
    train_loss, train_acc = train_metrics
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc


def updater(batch_size):
    return d2l.sgd([w, b], lr, batch_size)


lr = 0.1
num_epochs = 10
train_ch3(net, train_iter, test_iter, cross_entropy, num_epochs, updater)